## Simulation de Diffusion Thermique 2D

Cette simulation modélise la diffusion de la chaleur sur une surface bidimensionnelle. L'équation régissant ce phénomène est l'équation de la chaleur :

$$ \frac{\partial u}{\partial t} = \alpha \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right) $$

Où :
- $ u(x, y, t) $ est la température au point $ (x, y) $ au temps $ t $.
- $ \alpha $ est la diffusivité thermique du matériau.
- $ \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} $ est le Laplacien de la température, représentant la variation spatiale de la température.

Pour résoudre numériquement cette équation, nous utilisons une méthode de différences finies, spécifiquement la méthode FTCS (Forward Time Centered Space). L'équation discrétisée est :

$$ u_{i,j}^{n+1} = u_{i,j}^{n} + D \left( u_{i+1,j}^{n} + u_{i-1,j}^{n} + u_{i,j+1}^{n} + u_{i,j-1}^{n} - 4u_{i,j}^{n} \right) $$

Où :
- $ u_{i,j}^{n} $ est la température à la position discrète $ (i,j) $ au pas de temps $ n $.
- $ D = \frac{\alpha \Delta t}{(elta x)^2} $ est le nombre de Courant numérique. Pour la stabilité de la méthode FTCS en 2D, $ D \le 0.25 $ (en supposant $ \Delta x = \Delta y $).

Le terme $ (u_{i+1,j}^{n} + u_{i-1,j}^{n} + u_{i,j+1}^{n} + u_{i,j-1}^{n} - 4u_{i,j}^{n}) $ est une approximation discrète du Laplacien multiplié par $ (\Delta x)^2 $. Cela peut être calculé efficacement en utilisant une opération de convolution 2D avec le noyau Laplacien suivant :
$$ K = \begin{pmatrix} 0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0 \end{pmatrix} $$

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import Video, HTML

# Configuration pour l'animation (nécessaire dans certains environnements pour blitting)
plt.rcParams['animation.embed_limit'] = 2**128 # Augmenter la limite pour l'intégration de l'animation
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg' # Spécifiez le chemin si ffmpeg n'est pas dans le PATH

In [ ]:
# Paramètres de la simulation
N = 100  # Taille de la grille (N x N)
h = 1.0  # Pas spatial (supposé dx = dy = h)
alpha = 0.1  # Diffusivité thermique
dt = 1.0  # Pas de temps

# Vérification de la condition de stabilité
D_coeff = alpha * dt / (h**2)
print(f"Coefficient de stabilité D = {D_coeff:.4f}")
if D_coeff > 0.25:
    print("Attention : La simulation pourrait être instable avec ces paramètres.")
    print("Veuillez réduire dt ou alpha, ou augmenter h.")

num_frames = 200  # Nombre d'itérations (images dans la vidéo)

# Configuration du device (GPU si disponible, sinon CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du device : {device}")

In [ ]:
# Initialisation de la grille de température U
# U a la forme (batch_size, channels, height, width) pour conv2d
U = torch.zeros((1, 1, N, N), device=device, dtype=torch.float32)

# Condition initiale : un point chaud au centre
center_x, center_y = N // 2, N // 2
radius = 5
U[0, 0, center_x-radius:center_x+radius, center_y-radius:center_y+radius] = 20.0

# Noyau Laplacien
laplacian_kernel = torch.tensor([[
    [0, 1, 0],
    [1, -4, 1],
    [0, 1, 0]
]], device=device, dtype=torch.float32)
laplacian_kernel = laplacian_kernel.unsqueeze(0) # (out_channels, in_channels, kH, kW)

In [ ]:
def simulation_step(U_current):
    """Calcule un pas de temps de la simulation."""
    # Calcul du Laplacien en utilisant la convolution 2D
    # padding=1 assure que la taille de sortie est la même que l'entrée
    laplacian_U = F.conv2d(U_current, laplacian_kernel, padding=1)
    
    # Mise à jour de la température
    U_new = U_current + D_coeff * laplacian_U
    
    # Conditions aux limites de Dirichlet (température fixe aux bords)
    # Mettre à zéro les bords après la mise à jour
    U_new[:, :, 0, :] = 0.0  # Bord supérieur
    U_new[:, :, -1, :] = 0.0 # Bord inférieur
    U_new[:, :, :, 0] = 0.0  # Bord gauche
    U_new[:, :, :, -1] = 0.0 # Bord droit
    
    return U_new

In [ ]:
# Configuration de l'animation
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xticks([])
ax.set_yticks([])

# Affichage initial de la grille
# .cpu().numpy() pour convertir le tenseur PyTorch (potentiellement sur GPU) en array NumPy pour Matplotlib
img = ax.imshow(U[0, 0].cpu().numpy(), cmap=plt.cm.hot, vmin=0, vmax=20, animated=True)
fig.colorbar(img, ax=ax, label="Température (°C)")
ax.set_title("Diffusion de la chaleur")

# Copie de U pour l'animation, pour ne pas modifier l'original directement dans la boucle
U_anim = U.clone()

def animate(frame):
    global U_anim
    U_anim = simulation_step(U_anim)
    img.set_array(U_anim[0, 0].cpu().numpy())
    if frame % 20 == 0:
        print(f"Génération de l'image {frame}/{num_frames}")
    return img,

# Création de l'animation
# interval: délai entre les images en millisecondes
# blit=True optimise le redessinage
print("Début de la génération de l'animation...")
ani = animation.FuncAnimation(fig, animate, frames=num_frames, interval=50, blit=True)

# Sauvegarde de l'animation en fichier MP4
# Nécessite ffmpeg. S'il n'est pas dans le PATH, spécifiez plt.rcParams['animation.ffmpeg_path']
video_filename = 'thermal_diffusion_pytorch.mp4'
try:
    ani.save(video_filename, writer='ffmpeg', fps=20, dpi=100)
    print(f"Animation sauvegardée sous {video_filename}")
except FileNotFoundError:
    print("Erreur: ffmpeg n'a pas été trouvé. Veuillez l'installer et vous assurer qu'il est dans le PATH,")
    print("ou spécifiez son chemin avec plt.rcParams['animation.ffmpeg_path'].")
    # Fallback to HTML5 video if ffmpeg is not available
    # HTML(ani.to_html5_video())
    # print("Affichage en tant que vidéo HTML5.")
plt.close(fig) # Ferme la figure pour éviter l'affichage statique en double

In [ ]:
# Affichage de la vidéo dans le notebook
try:
    # Vérifier si le fichier vidéo a été créé
    with open(video_filename, 'rb') as f:
        pass # Le fichier existe
    display(Video(video_filename, embed=True))
except FileNotFoundError:
    print(f"Le fichier vidéo {video_filename} n'a pas été trouvé. L'affichage est impossible.")
    print("Cela peut être dû à un échec de la sauvegarde (ex: ffmpeg manquant).")